<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GeneticAlgorithmTSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
np.set_printoptions(formatter={'float':lambda x: f'{x:5.2f}'}, linewidth=240)

In [2]:
def plot_single(seq):
    fig=make_subplots()
    fig.add_scatter(x=coords[:,0],y=coords[:,1],mode='markers')
    fig.add_scatter(x=coords[seq,0],y=coords[seq,1],mode='lines')
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.update_layout(width=600, template='plotly_dark',showlegend=False)
    fig.show()

def plot(seqs,dists,nr=4,nc=4):
    fig=make_subplots(rows=nr,cols=nc, shared_xaxes=True, shared_yaxes=True)
    fig.add_scatter(x=coords[:,0], y=coords[:,1],mode='markers')
    s=np.r_[seqs, np.atleast_2d(seqs[0,:])]
    for i in range(seqs.shape[1]):
        fig.add_scatter(x=coords[:,0],y=coords[:,1],mode='markers', row=i//nc+1,col=i%nc + 1, marker_color='rgba(0,0,0,0)', marker_line_color='green', marker_line_width=1)
        fig.add_scatter(x=coords[s[:,i],0],y=coords[s[:,i],1],mode='lines',row=i//nc+1,col=i%nc + 1)
        fig.add_annotation(x=10,y=12,text=f'{dists[i]:.2f}', xanchor='right',showarrow=False, row=i//nc+1,col=i%nc + 1)
    fig.update_yaxes(scaleanchor = "x", scaleratio = 1)
    fig.update_layout(width=nc*200, height=nr*200,  template='plotly_dark',showlegend=False)
    fig.show()


In [3]:
def calcdist(seq):
    N=seq.shape[0]
    return np.sum(dist[seq[1:,:],seq[:-1,:]],axis=0)+ dist[seq[0,:], seq[-1,:]]

In [4]:
def mutate(seq, n):
    N, B = seq.shape
    i = np.argsort(rng.uniform(size=(N-1,B)),axis=0)
    temp=np.take_along_axis(seq,i[:n],axis=0)
    np.put_along_axis(seq,i[:n], np.take_along_axis(seq,i[n:2*n], axis=0),axis=0)
    np.put_along_axis(seq,i[n:2*n], temp,axis=0)

In [5]:
def mate(seq):

    parents = seq[:,rng.permutation(range(seq.shape[1]))]
    diff = parents[:,::2]!=parents[:,1::2]
    children = parents.copy()
    children[:,::2][diff]=children[:,1::2][diff]
    children[:,1::2][diff]=children[:,::2][diff]
    mutate_idx = rng.randint(1,children.shape[1],size=max(int(0.8*children.shape[1]),int(0.2*N)))
    mutate_children = children[:,mutate_idx]
    mutate(mutate_children,1)
    children[:,mutate_idx]=mutate_children
    return children

In [6]:
def topn(pop, n, unique_only=False):
    global bestdist, bestseq
    seq=pop
    if unique_only:
        seq=np.unique(pop,axis=1)
    dists=calcdist(seq)
    topidx=np.argsort(dists)[:n]
    return seq[:,topidx]

In [7]:
N=100
rng=np.random.RandomState(1344)
coords = rng.uniform(0,10,size=(N,2))
coords = rng.permutation(coords)
dist=np.sqrt(np.sum((coords[:,None,:] - coords[None,:,:])**2,axis=-1))

In [9]:
#initial route
rng=np.random.RandomState(13434)
popsize=1024
Ninit=popsize
pop = np.zeros((N,Ninit), dtype=np.int16)

for j in range(Ninit):
    rem = list(range(1,N))
    pop[0,j]=rng.randint(N)
    cur=pop[0,j]
    for i in range(1,N-1):
        best2 = np.argsort(dist[cur][rem])[:2]
        nxt=rem[best2[rng.choice([0,1],p=(0.95,0.05))]]
        rem.remove(nxt)
        pop[i,j]=nxt
        cur=nxt
    pop[-1,j]=rem[0]


pop = np.tile(pop,(1,(popsize//Ninit)+1))[:,:popsize]


In [10]:

N_gen=500
for _ in range(N_gen):
    better_pop =topn(pop,pop.shape[1]//2,unique_only=False)
    pop=np.concatenate([better_pop, mate(better_pop)],axis=1)
    

In [11]:
top16=topn(pop,50,unique_only=True)
plot(top16,calcdist(top16),nr=5,nc=10)

In [ ]:
top16.shape